In [2]:
# https://www.programcreek.com/python/example/100588/keras.layers.normalization.BatchNormalization

In [5]:

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.noise import AlphaDropout
from keras.utils import np_utils
from keras.optimizers import RMSprop
from keras.initializers import lecun_uniform
from keras.wrappers.scikit_learn import KerasClassifier

def construct_model(layers=[50,50], activation='relu', a_reg='l1', b_reg='l1', b_init='random_uniform', k_init='random_uniform', dropout=.0,\
                    a_dropout=.0, optimizer='sgd', loss='binary_crossentropy', metrics='roc_auc_score',input_dim=17, output_dim=1, model_name='most'):
    """
    Helper to construct a Keras model based on dict of specs and input size

    Parameters
    ----------
    model_spec: dict
        Dict containing keys: arch, activation, dropout, optimizer, loss,
            w_reg, metrics
    input_dim: int
        Size of input dimension
    output_dim: int
        Size of input dimension

    Returns
    -------
    model: Compiled keras.models.Sequential

    """

    model = Sequential()

    for li, layer_size in enumerate(layers):
        # For input layer, add input dimension
        if li == 0:
            temp_input_dim = input_dim
            model.add(Dense(layer_size,
                            input_dim=input_dim,
                            activation=activation,
                            activity_regulizer=model_spec['a-reg'],
                            bias_regulizer=model_sepc['b-reg'],
                            bias_initializer=model_spec['b-init'],
                            kernel_initializer=model_spec['k-init'],
                            name='Input'))
        elif li < len(layers)-1:
            model.add(Dense(layer_size,
                            activation=model_spec['activation'],
                            activity_regulizer=model_spec['a-reg'],
                            bias_regulizer=model_spec['b-reg'],
                            bias_initializer=model_spec['b-init'],
                            kernel_initializer=model_spec['k-init'],
                            name='Layer_%i' % li))
        elif li == len(layers)-1:
            model.add(Dense( output_dim ,
                            activation='sigmoid',
                            activity_regulizer=model_spec['a-reg'],
                            bias_regulizer=model_spec['b-reg'],
                            bias_initializer=model_spec['b-init'],
                            kernel_initializer=model_spec['k-init'],
                            name='Layer_%i' % li))
            
        if model_spec['dropout'] > 0.:
            model.add(Dropout(model_spec['dropout'], name='Dropout_%i' % li))
        if model_spec['a-dropout'] > 0.:
            model.add(Dropout(model_spec['a-dropout'], name='Dropout_%i' % li))
    model.compile(optimizer=model_spec['optimizer'],
                  loss=model_spec['loss'],
                  metrics=model_spec['metrics'])
    return model 
model = KerasClassifier(build_fn=construct_model, epochs=100, batch_size=32,\
                       verbose=0)

Using TensorFlow backend.


In [7]:
#custom scoring function sklearn
from sklearn.grid_search import GridSearchCV
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
def lift_index(y, probabilities):
    #rearrange probabilities and y in descending order
    combine = zip(y, probabilities)
    dtype = [('data', int), ('prob', float)]
    combine = np.array(combine,dtype=dtype)
    combine = np.sort(combine, order='prob')
    ordered = np.sort(probabilities)
    #divide into deciles
    deciles = pd.qcut(ordered, 10)
    #amount of positive in each decile
    i=0
    sum_deciles = sum([sum([((num/10)+0.1)*combine[i][0] for i in deciles if i==num ]) for num in range(10)])
    return sum_deciles/sum(y)
lift_index_score = make_scorer(lift_index, greater_is_better=True)
    

C:\Users\Snir\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Snir\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [8]:

def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [11]:
import matplotlib as plt
from matplotlib import cm

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        title='Normalized confusion matrix'
    else:
        title='Confusion matrix'

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
#fix seed for reproducibility
seed=7
np.random.seed(seed)
#import data without any dummy variables or changes
param_grid = {'dense_layers': [[100,100],[100,100,100],[50,50,50,50]],
              'activation':['relu','selu'],
              'dropout': [0.0, 0.5],
              'a-dropout': [0.0,0.2],
              'k-init': ['lecun_normal'],
              'b-init': ['lecun_normal'],
              'a-reg': ['l1'],
              'b-reg': ['l2'],
              'optimizer':('rmsprop','adam'),
              'epochs':[10,50,100],
              'batch_size':[128,32,16]}
#improve with article
grid = GridSearchCV(model,
                    param_grid=param_grid,
                    return_train_score=True,
                    scoring=[lift_index_score,'roc_auc_score'],
                    refit='roc_auc_score',
                    n_jobs=-1)

#grid_results = grid.fit(x_train,y_train)
#means = grid_result.cv_results_['mean_test_score']
#stds = grid_result.cv_results_['std_test_score']
#params = grid_result.cv_results_['params']
#for mean, stdev, param in zip(means, stds, params):
#    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#visualization
from keras.utils import plot_model
#visualize best model
# use params from prev cell to visualize
#visualize
plot_model(model, to_file='model_%.png' % model_name)
